# Data extraction

The goal of this project is to extract relevant data from text files, previously converted from PDF files. Since the text files are quite structured, the decision is to use regex to do this.

In [72]:
from pathlib import Path
from pprint import pprint
from typing import List, Tuple
from collections import defaultdict
import re
import pandas as pd
import re
from tqdm import tqdm
import numpy as np

from datetime import datetime
import shutil

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', None)

Define paths:

In [73]:
RAW_DATA_PATH = Path('/Users/nenadbozinovic/Documents/regex/RR Procurement - Raw Data')
RESULTS_PATH = RAW_DATA_PATH.parent / 'results'
RESULTS_PATH.mkdir(exist_ok=True, parents=True)

OUTLIERS_PATH = RESULTS_PATH / 'outliers'

RAW_DATA_PATH_PDF = RAW_DATA_PATH / 'PDFs'
OUTLIERS_PATH_PDF = OUTLIERS_PATH / 'PDFs'
OUTLIERS_PATH_PDF.mkdir(exist_ok=True, parents=True)

RAW_DATA_PATH_LINEPRINTER = RAW_DATA_PATH / 'Txt files - lineprinter'
OUTLIERS_PATH_LINEPRINTER = OUTLIERS_PATH / 'Txt files - lineprinter'
OUTLIERS_PATH_LINEPRINTER.mkdir(exist_ok=True, parents=True)


Define column names so we don't use literals at any point:

In [32]:

IDENTIFIER = "Identifier"
POSTPONED_CONTRACT = "Postponed_Contract"
NUMBER_OF_BIDDERS = "Number_of_Bidders"
BID_OPENING_DATE = "Bid_Opening_Date"
CONTRACT_DATE = "Contract_Date"
CONTRACT_NUMBER = "Contract_Number"
TOTAL_NUMBER_OF_WORKING_DAYS = "Total_Number_of_Working_Days"
CONTRACT_ITEMS = "Number_of_Contract_Items"
CONTRACT_DESCRIPTION = "Contract_Description"
PERCENT_OVER_EST = "Percent_Est_Over"
PERCENT_UNDER_EST = "Percent_Est_Under"
ENGINEERS_EST = "Engineers_Est"
AMOUNT_OVER = "Amount_Over"
AMOUNT_UNDER = "Amount_Under"
CONTRACT_CODE = "Contract_Code"

BID_RANK = "Bid_Rank"
BID_TOTAL = "Bid_Total"   
BIDDER_ID = "Bidder_ID"
BIDDER_NAME = "Bidder_Name"
BIDDER_NAME_COND = "Bidder_Name_cond"
CSLB_NUMBER = "CSLB_Number"

SUBCONTRACTOR_NAME = "Subcontractor_Name"
SUBCONTRACTED_LINE_ITEM = "Subcontracted_Line_Item"

ITEM_NUMBER = "Item_Number"
ITEM_CODE = "Item_Code"
ITEM_DESCRIPTION = "Item_Description"
ITEM_DOLLAR_AMOUNT = "Item_Dollar_Amount"

CITY = "City"
LICENSE_NUMBER = "License_Number"


In [74]:
def get_contract_number_and_tag_from_filename(filename:str) -> Tuple[str, str]:
    pattern = re.compile(r"^(\d{2}-\w+)\.pdf_(\d+)$", re.IGNORECASE)  # IGNORECASE is critical since names might have both PDF and pdf
    match = pattern.search(filename)
    contract_number, tag = match.groups()
    identifier = f"{contract_number}_{tag}"
    return contract_number, tag, identifier


def get_contract_number(file_contents):
    return extract(file_contents, r"CONTRACT NUMBER\s+([A-Za-z0-9-]+)")


def get_dates(file_contents):
    match = re.search(r"BID OPENING DATE\s+(\d+\/\d+\/\d+).+\s+(\d+\/\d+\/\d+)", file_contents)
    return match.group(1), match.group(2)
    


def extract(file_contents, regex, post_processing=None):
    # Search for the pattern in the text
    match = re.search(regex, file_contents)

    if match:
        # Extract first capture group
        if post_processing:
            temp = post_processing(match.group(1))
        else:
            temp = match.group(1)
        return temp
    else:
        return ""
    

def extract_contract_data(file_contents, identifier):
    row = defaultdict(str)
    row[IDENTIFIER] = identifier
    match = extract(file_contents, r"(POSTPONED CONTRACT)")
    row[POSTPONED_CONTRACT] = 1 if match else 0
    row[BID_OPENING_DATE], row[CONTRACT_DATE] = get_dates(file_contents)
    row[CONTRACT_CODE] = extract(file_contents, r"CONTRACT CODE\s+'([^']+)'")
    row[CONTRACT_ITEMS] = extract(file_contents, r"(\d+)\s+CONTRACT ITEMS")
    row[TOTAL_NUMBER_OF_WORKING_DAYS] = extract(file_contents, r"TOTAL NUMBER OF WORKING DAYS\s+(\d+)")
    row[NUMBER_OF_BIDDERS] = extract(file_contents, r"NUMBER OF BIDDERS\s+(\d+)")
    row[ENGINEERS_EST] = extract(file_contents, r"ENGINEERS EST\s+([\d,]+\.\d{2})")
    row[AMOUNT_OVER] = extract(file_contents, r"AMOUNT OVER\s+([\d,]+\.\d{2})")
    row[AMOUNT_UNDER] = extract(file_contents, r"AMOUNT UNDER\s+([\d,]+\.\d{2})")
    row[PERCENT_OVER_EST] = extract(file_contents, r"PERCENT OVER EST\s+(\d+.\d{2})")
    row[PERCENT_UNDER_EST] = extract(file_contents, r"PERCENT UNDER EST\s+(\d+.\d{2})")
    row[CONTRACT_DESCRIPTION] = extract(file_contents, r"(?:\n)?(.*?)FEDERAL AID", post_processing=lambda x: x.lstrip())
    return row


def extract_contract_bid_data(file_contents, identifier):

    # have fixed width for name (37 characters) and CSLB number (8 digits)
    pattern = re.compile(r"(\d+)\s+(A\))?\s+([\d,]+\.\d{2})\s+(\d+)\s+(.{37})\s(\d{3} \d{3}-\d{4})(.*)?$\s+(.*?)(.{37})\s(\d{8})", re.MULTILINE)
    matches = pattern.findall(file_contents)
    
    contract_bid_data = []

    for match in matches:
        row = defaultdict(str)
        row[IDENTIFIER] = identifier
        row[BID_RANK] = match[0]
        row["OPTIONAL_A"] = match[1]
        row[BID_TOTAL] = match[2]
        row[BIDDER_ID] = match[3].strip()
        row[BIDDER_NAME] = match[4].strip()
        row["BIDDER PHONE"] = match[5].strip()
        row["Extra_Text"] = match[6]
        row['Weird_Contract_Notes'] = match[7]
        row[BIDDER_NAME] += ' ' + match[8]
        row[BIDDER_NAME].strip()
        row[CSLB_NUMBER] = match[9] 
        contract_bid_data.append(row)
        
    return contract_bid_data


def reduce_file_contents(file_contents):
    # here we first extract file_contents_relevant from file_contents so we narrow down the search
    pattern = re.compile(r"(?s)BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED(.*?)(?=BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED|\f|CONTINUED ON NEXT PAGE)")
    matches = pattern.findall(file_contents)
    return matches


def extract_bid_subcontractor_data(file_contents, identifier):
    """
    First we get the relevant information from a whole contract using this:
"BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED(.*?)(?=BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED|\f|CONTINUED ON NEXT PAGE)"
this means starting phrase must be X, then text that we want extracted and then the match can either finish with X, Y or Z.
In our case:
X = BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED
Y = \f (this is a new page character, in the text is denoted as FF, but this is not a pure FF text but /f)
Z = CONTINUED ON NEXT PAGE

I also ensure that we are doing loopback so the matched do not overlap with ?=

Now that we have a table we exact the columns, I use some fixed with columns for that:
"^\s+(\d{2})?\s+(.{60})\s+(.+)\n\s+(.{40})(\d+)\n"

    """
    
    pattern = re.compile(r"(?s)BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED(.*?)(?=BIDDER ID NAME AND ADDRESS\s+LICENSE NUMBER\s+DESCRIPTION OF PORTION OF WORK SUBCONTRACTED|\f|CONTINUED ON NEXT PAGE)")
    matches = pattern.findall(file_contents)
    if not matches:
        return []
            
    bid_subcontractor_data = []
    for match in matches:
        pattern2 = re.compile(r"(?m)^\s+(\d{2})?\s+(.{60})\s+(.+)\n\s+(.{40})(\d+)\n")
        matches2 = pattern2.findall(match)
        
        for match2 in matches2:
            row = defaultdict(str)
            row[IDENTIFIER] = identifier
            row[BIDDER_ID] = match2[0]
            row[SUBCONTRACTOR_NAME] = match2[1]
            row[SUBCONTRACTED_LINE_ITEM] = match2[2]
            row[CITY] = match2[3]
            row[LICENSE_NUMBER] = match2[4]
            
            bid_subcontractor_data.append(row)
        
    return bid_subcontractor_data


def extract_contract_line_item_data(file_contents, identifier):

    pattern = re.compile(r"(^\s*(\d+)\s+(\d+)\s+([\dA-Z\(\)\"\- ]{46})\s(.{35})\s+([\d,]+\.\d{2}))(?:\n\s+([\dA-Z\(\)\"\- $]+)\n)?", re.MULTILINE)

    # TODO use fixed width approach from extract_bid_subcontractor_data
    # and extract first text between dotted line and total: 
    # ^\s+-+\n([\s\S]+?)\n\s+TOTAL\s+\$?([\d,]+\.\d{2})
    matches = pattern.findall(file_contents)

    contract_line_item_data = []
    for match in matches:
        row = defaultdict(str)
        row[IDENTIFIER] = identifier
        row[ITEM_NUMBER] = match[1]
        row[ITEM_CODE] = match[2]
        row[ITEM_DESCRIPTION] = match[3].rstrip() + ' ' + match[6]
        row[ITEM_DOLLAR_AMOUNT] = match[5]
        contract_line_item_data.append(row)
        
    return contract_line_item_data


def write_to_results(data: List, name: str):
    df = pd.DataFrame(data)
    df.to_csv(RESULTS_PATH / f'{name}.csv', index=False)
    

def read_file(filepath: str):
    # Open the file in read mode ('r')
    with open(filepath, 'r') as file:
        # Read the contents of the file into a string
        file_contents = file.read()
    return file_contents

<>:88: SyntaxWarning: invalid escape sequence '\s'
<>:88: SyntaxWarning: invalid escape sequence '\s'
/var/folders/9f/w9g4yl_d2x16ynkjk3tmwpph0000gn/T/ipykernel_4603/21494857.py:88: SyntaxWarning: invalid escape sequence '\s'
  """


# One sample study

In [37]:
# filepath = RAW_DATA_PATH.parent / 'sample' / '01-0A3804.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A3804.pdf_4353.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A0904.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A1204.pdf_11468.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0F4304.pdf_12346.txt'  # issue # 11
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K6104.pdf_12731.txt'  # issue # 9
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0H3204.pdf_9871.txt'  # issue # 2

filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K4604.pdf_12040.txt'  # issue # 1


In [38]:
contract_number_from_filename, tag, identifier = get_contract_number_and_tag_from_filename(filepath.stem)

file_contents = read_file(filepath)

In [39]:
contract_number_from_filename, tag, identifier 

('01-0K4604', '12040', '01-0K4604_12040')

Extract contract data:

In [75]:
df_contract_data = pd.DataFrame([extract_contract_data(file_contents, identifier)])
df_contract_bid_data = pd.DataFrame(extract_contract_bid_data(file_contents, identifier))
df_bid_subcontractor_data = pd.DataFrame(extract_bid_subcontractor_data(file_contents, identifier))
df_contract_line_item_data = pd.DataFrame(extract_contract_line_item_data(file_contents, identifier))

In [76]:
df_contract_data

,Identifier,Postponed_Contract,Bid_Opening_Date,Contract_Date,Contract_Code,Number_of_Contract_Items,Total_Number_of_Working_Days,Number_of_Bidders,Engineers_Est,Amount_Over,Amount_Under,Percent_Est_Over,Percent_Est_Under,Contract_Description
0,01-0K4604_12040,0,05/05/21,05/07/21,A,23,40,7,"2,987,774.00",,"1,038,198.60",,34.75,


In [77]:
df_contract_bid_data

,Identifier,Bid_Rank,OPTIONAL_A,Bid_Total,Bidder_ID,Bidder_Name,BIDDER PHONE,Extra_Text,Weird_Contract_Notes,CSLB_Number
0,01-0K4604_12040,1,,"1,949,575.40",7,"ALL-AMERICAN CONSTRUCTION, INC. ...",530 617-5111,SB PREF CLAIMED,,00948762
1,01-0K4604_12040,2,,"2,099,133.00",6,"O.C. JONES & SONS, INC. ...",510 526-3424,CC PREF CLAIMED,,00759729
2,01-0K4604_12040,3,,"2,179,226.00",4,BALDWIN CONTRACTING COMPANY INC. DBA. KNIFE RI...,530 891-6555,CC PREF CLAIMED,,00091712
3,01-0K4604_12040,4,,"2,205,071.00",5,ARGONAUT CONSTRUCTORS ...,707 542-4862,CC PREF CLAIMED,,00171432
4,01-0K4604_12040,5,,"2,282,828.00",3,GHILOTTI CONSTRUCTION CO. ...,707 585-1221,NSB PREF CLAIMED,,00644515
5,01-0K4604_12040,6,,"2,381,842.00",1,GRANITE CONSTRUCTION COMPANY ...,707 467-4100,NSB PREF CLAIMED,,00000089
6,01-0K4604_12040,7,,"2,785,186.50",2,"S.T.RHOADES CONSTRUCTION, INC. ...",530 223-9322,CC PREF CLAIMED,,00930684


In [78]:
df_bid_subcontractor_data['Bidder_ID'] = df_bid_subcontractor_data['Bidder_ID'].replace('', np.nan)
df_bid_subcontractor_data['Bidder_ID'] = df_bid_subcontractor_data['Bidder_ID'].ffill()
df_bid_subcontractor_data

,Identifier,Bidder_ID,Subcontractor_Name,Subcontracted_Line_Item,City,License_Number
0,01-0K4604_12040,07,ANRAK CORPORATION ...,"CUT, LOAD, SWEEP & HAUL GRINDINGS",SACRAMENTO CA,256390
1,01-0K4604_12040,07,ANRAK CORPORATION ...,ITEM 10 (17%),SACRAMENTO CA,256390
2,01-0K4604_12040,07,ANRAK CORPORATION ...,ITEM 15 (100%),SACRAMENTO CA,256390
3,01-0K4604_12040,07,CENTRAL STRIPING SERVICE INC. ...,ITEM 1722 (100%),RANCHO CORDOVA CA,403896
4,01-0K4604_12040,07,CENTRAL STRIPING SERVICE INC. ...,ITEM 23 (50%),RANCHO CORDOVA CA,403896
5,01-0K4604_12040,07,CENTRAL STRIPING SERVICE INC. ...,ITEM 4 (15%),RANCHO CORDOVA CA,403896
6,01-0K4604_12040,07,CENTRAL STRIPING SERVICE INC. ...,RUMBLE STRIPS GRIND ONLY,RANCHO CORDOVA CA,403896
7,01-0K4604_12040,07,CENTRAL STRIPING SERVICE INC. ...,TRAFFIC CONTROL PARTIAL STRIPING,RANCHO CORDOVA CA,403896
8,01-0K4604_12040,07,PACIFIC NORTHWEST OIL ...,ITEM 12 (100%),SACRAMENTO CA,705296
9,01-0K4604_12040,07,PACIFIC NORTHWEST OIL ...,ITEM 13 (100%),SACRAMENTO CA,705296


In [79]:
df_contract_line_item_data

,Identifier,Item_Number,Item_Code,Item_Description,Item_Dollar_Amount
0,01-0K4604_12040,1,070030,LEAD COMPLIANCE PLAN,"2,450.00"
1,01-0K4604_12040,2,090100,TIME-RELATED OVERHEAD (WDAY),"24,000.00"
2,01-0K4604_12040,3,120090,CONSTRUCTION AREA SIGNS,"2,200.00"
3,01-0K4604_12040,4,120100,TRAFFIC CONTROL SYSTEM,"53,000.00"
4,01-0K4604_12040,5,011484,PORTABLE RADAR SPEED FEEDBACK SIGN SYSTEMS,"2,400.00"
5,01-0K4604_12040,6,128652,PORTABLE CHANGEABLE MESSAGE SIGN (LS),"3,200.00"
6,01-0K4604_12040,7,130100,JOB SITE MANAGEMENT,"19,500.00"
7,01-0K4604_12040,8,130200,PREPARE WATER POLLUTION CONTROL PROGRAM,695.00
8,01-0K4604_12040,9,190185,SHOULDER BACKING,"52,000.00"
9,01-0K4604_12040,10,390095,REPLACE ASPHALT CONCRETE SURFACING,"535,600.00"


# Batch run

Here we run a batch or a single file (for example, if you want to run a specific outlier, make `files` a single element list):

In [15]:
files = (RAW_DATA_PATH/'Txt files - lineprinter').glob('*.txt')
# files = [RAW_DATA_PATH/'Txt files - lineprinter'/'01-0F9204.PDF_12364.txt']

In [16]:
contract_data = []
contract_bid_data = []
bid_subcontractor_data = []
contract_line_item_data = []
outliers = []

for filepath in tqdm(files):
    
    file_contents = read_file(filepath)
        
    filename = filepath.stem
    contract_number_from_filename, tag, identifier = get_contract_number_and_tag_from_filename(filename)
    contract_number_from_contents = get_contract_number(file_contents)
    
    if contract_number_from_filename == contract_number_from_contents:  
        contract_data.append(extract_contract_data(file_contents, identifier))
        contract_bid_data.extend(extract_contract_bid_data(file_contents, identifier))
        bid_subcontractor_data.extend(extract_bid_subcontractor_data(file_contents, identifier))
        contract_line_item_data.extend(extract_contract_line_item_data(file_contents, identifier))
    else:
        # if contract number doesn't match then something is off that needs investigation
        outliers.append({'outlier_filename': filename})
        # let's also copy the pdf to a folder for manual inspection
        source_path = RAW_DATA_PATH_PDF / f'{filename}.pdf'
        destination_path = OUTLIERS_PATH_PDF / f'{filename}.pdf'
        shutil.copy(source_path, destination_path)
        
        source_path = RAW_DATA_PATH_LINEPRINTER / f'{filename}.txt'
        destination_path = OUTLIERS_PATH_LINEPRINTER / f'{filename}.txt'
        shutil.copy(source_path, destination_path)

0it [00:00, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
5it [00:00, 30.11it/s]0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
370it [00:35, 10.54it/s]


# Save files

In [17]:
# # one can add timestamp to the files if needed
# timestamp = datetime.strftime(datetime.now(), '%m-%d-%Y-%H:%M:%S')
write_to_results(contract_data, "contract_data")
write_to_results(contract_bid_data, "contract_bid_data")
write_to_results(bid_subcontractor_data, "bid_subcontractor_data")
write_to_results(contract_line_item_data, "contract_line_item_data")
write_to_results(outliers, "outliers")

# Save to excel

In [18]:
import pandas as pd

# Paths to your CSV files
csv_file_paths = RESULTS_PATH.glob('*.csv')

# Path to the output Excel file
excel_file_path = RESULTS_PATH / 'results.xlsx'

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    # Iterate over your CSV files
    for csv_file in csv_file_paths:
        # Use Path from pathlib to work with file paths
        csv_path = Path(csv_file)
        
        # Extract the file name without the extension for the sheet name
        sheet_name = csv_path.stem
        
        # Read each CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Write the DataFrame to a new sheet in the Excel file using the file name as the sheet name
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f'Merged CSV files into {excel_file_path}')

Merged CSV files into /Users/nenadbozinovic/Documents/regex/results/results.xlsx
